In [7]:
%pip install geopandas tobler pydot seaborn networkx

import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import fiona
import shapely
import seaborn  as sns
from scipy.optimize import curve_fit
import tobler
import matplotlib
import networkx as nx
from shapely.geometry import Point, MultiPoint
from shapely.ops import nearest_points
from scipy.spatial import cKDTree
import pydot
from networkx.drawing.nx_pydot import graphviz_layout
from matplotlib import colors
from matplotlib import cm
import random
import pickle
import math
import ast
from shapely.geometry import Point, LineString
from scipy.spatial import cKDTree
from scipy.optimize import minimize
from scipy.integrate import odeint


Note: you may need to restart the kernel to use updated packages.


In [8]:
plt.rcParams.update({'font.size': 16})

#### Import Cassini map

In [13]:
# Download data from: http://dx.doi.org/10.7910/DVN/28674
edges=gpd.read_file("../Maps/edge/edge.shp")
edges=edges.set_crs("EPSG:2154")


DataSourceError: ../Maps/edge/edge.shp: No such file or directory

In [10]:
# Download data from: http://dx.doi.org/10.7910/DVN/28674
nodes=gpd.read_file("../Maps/node/node.shp")
nodes=nodes.set_crs("EPSG:2154")


DataSourceError: ../Maps/node/node.shp: No such file or directory

In [ ]:
nodes1=nodes[nodes.component==1]

In [ ]:
cities=nodes.dropna()

In [ ]:
cities_joined=cities.dissolve(by="city_name").reset_index()

In [ ]:
cities_joined.geometry=cities_joined.geometry.representative_point()

In [ ]:
G_roads=nx.from_pandas_edgelist(edges, source='start_node', target='end_node', edge_attr="length")

In [ ]:
pos = {row["node_id"]: (row["geometry"].x, row["geometry"].y) for _, row in nodes.iterrows()}

In [ ]:
nx.set_node_attributes(G_roads, pos, "pos")

In [ ]:
# Convert positions to array format for KDTree
positions = np.array([pos[node] for node in G_roads.nodes()])
node_ids = list(G_roads.nodes())

In [ ]:
# Create a KDTree for fast neighbor lookup
tree = cKDTree(positions)


In [ ]:
threshold=1000.

In [ ]:
# Find pairs of points within the threshold
pairs = tree.query_pairs(threshold)


In [ ]:
for i, j in pairs:
    node1 = node_ids[i]
    node2 = node_ids[j]
    distance = np.linalg.norm(positions[i] - positions[j])  # Optional: calculate exact distance
    G_roads.add_edge(node1, node2, length=distance)

#### Load outline of France 


In [ ]:
fr_outline=gpd.read_file("../Maps/france_outline.geojson")

#### Load transmission network for the Great Fear

In [ ]:
with open("../Data/G_tot.p", "rb") as f:
    G_tot = pickle.load(f)

In [ ]:
edge_data = list(G_tot.edges(data=True))

In [ ]:
node_data = list(G_tot.nodes(data=True))

In [ ]:
node_data = []
for n, attr in G_tot.nodes(data=True):
    if 'location' in attr and isinstance(attr['location'], tuple):
        # Include the node name and all attributes
        node_data.append({**attr, "node": n})
    else:
        print(f"Warning: Node {n} does not have a valid location attribute.")



In [ ]:
geometry=[Point(attr['location']) for attr in node_data]

In [ ]:
df_node = pd.DataFrame.from_dict(dict(G_tot.nodes(data=True)), orient='index')

In [ ]:
nodes_gdf = gpd.GeoDataFrame(
    node_data,
    geometry=[Point(d['location']) for d in node_data],
    crs="EPSG:2154"  # Set CRS to EPSG:3035
)


#### Compute distances between nodes

In [ ]:
def euclidean_distance(coord1, coord2):
    """
    Calculate the Euclidean distance between two points in EPSG:3035 coordinates.
    Parameters:
        coord1: tuple (x, y) in meters
        coord2: tuple (x, y) in meters
    Returns:
        Distance in meters
    """
    dx = coord2[0] - coord1[0]
    dy = coord2[1] - coord1[1]
    return np.sqrt(dx**2 + dy**2)

In [ ]:
for u, v in G_tot.edges():
    loc_u = G_tot.nodes[u]['location']
    loc_v = G_tot.nodes[v]['location']
    distance = euclidean_distance(loc_u, loc_v)  # Compute distance
    G_tot[u][v]['distance'] = distance/1000  # Add as edge attribute

#### Compute travel times

In [ ]:
for u, v in G_tot.edges():
    time_v = G_tot.edges[u,v].get("day")
    time_u1 = G_tot.nodes[u]['day']
    time_u2 = G_tot.nodes[u]['day']
    travel_time1=time_v-time_u1
    travel_time2=time_v-time_u1
    travel_time=min(travel_time1,travel_time2)
    if travel_time<0:
        travel_time=np.nan
    if travel_time==0:
        time_u=(G_tot.nodes[u]['Time_min']+G_tot.nodes[u]['Time_max'])/2
        time_v=(G_tot.nodes[u]['Time_min']+G_tot.nodes[u]['Time_max'])/2
        travel_time=(time_v-time_u)/24
        if travel_time==0:
#            travel_time=0.5
            travel_time=np.nan
    G_tot[u][v]['travel_time'] = travel_time  # Add as edge attribute

#### Compute velocities

In [ ]:
for u, v in G_tot.edges():
    dist=G_tot[u][v]['distance']  
    time=G_tot[u][v]['travel_time']
    velocity=dist/(time)
    G_tot[u][v]['velocity'] = velocity 

#### Compute out degrees

In [ ]:
node_degrees = dict(G_tot.out_degree())

In [ ]:
nx.set_node_attributes(G_tot, node_degrees, "degree")

#### Create edge dataframe

In [ ]:
df_edge = pd.DataFrame(
    ({"source": u, "target": v, **attr} for u, v, attr in G_tot.edges(data=True))
)

In [ ]:
def ckdnearest(gdA, gdB, distname):
    nA = np.array(list(gdA.geometry.apply(lambda x: (x.x, x.y))))
    nB = np.array(list(gdB.geometry.apply(lambda x: (x.x, x.y))))
    btree = cKDTree(nB)
    dist, idx = btree.query(nA, k=1)
    gdB_nearest = gdB.iloc[idx].drop(columns="geometry").reset_index(drop=True)
    gdf = pd.concat(
        [
            gdA.reset_index(drop=True),
            gdB_nearest,
            pd.Series(dist, name=distname)
        ], 
        axis=1)

    return gdf

In [ ]:
nodes_gdf2=ckdnearest(nodes_gdf, nodes1, "dist0")

In [ ]:
dic_node_id=dict(zip(nodes_gdf2.node,nodes_gdf2.node_id))
dic_dist=dict(zip(nodes_gdf2.node,nodes_gdf2.dist0))

In [ ]:
df_edge["road_distance"]=0

#### Compute road distances

In [ ]:
for index, row in df_edge.iterrows():
    source=row["source"]
    target=row["target"]
    source_id=dic_node_id.get(source)
    target_id=dic_node_id.get(target)
    dist_s=dic_dist.get(source)/1000
    dist_t=dic_dist.get(target)/1000
    try:
        distance=nx.shortest_path_length(G_roads, source=source_id, target=target_id, weight="length", method='dijkstra')/1000.
        df_edge.at[index, "road_distance"] = distance + dist_s + dist_t
    except nx.NetworkXNoPath:
        df_edge.at[index, "road_distance"] = df_edge.at[index, "distance"]



In [ ]:
# Extract 'day' values for coloring the nodes
node_days = [G_tot.nodes[node]['day'] for node in G_tot.nodes]
# Convert the 'day' attribute to numeric, forcing errors to NaN
node_days = pd.to_numeric(node_days, errors='coerce')
node_colors = [
    'grey' if np.isnan(day) else day
    for day in node_days
]

# Normalize the 'day' values to [0, 1] range for colormap (ignoring 'grey' nodes)
valid_days = [day for day in node_colors if day != 'grey']
norm = colors.Normalize(vmin=min(valid_days), vmax=max(valid_days)) if valid_days else colors.Normalize(vmin=0, vmax=1)


##### Figure 1c

In [ ]:
# Choose the colormap (e.g., 'viridis' or any other from matplotlib)
cmap = cm.plasma

# Get the colors for each node based on the normalized 'day' value
node_color = [
    cmap(norm(day)) if day != 'grey' else 'grey'
    for day in node_colors
]


In [ ]:
colors3=cmap(np.linspace(0, 1, 18))

In [ ]:
fig = plt.figure(figsize=(8,8))
ax = fig.add_axes([0, 0, 1, 1])
ax.axis('off')
fr_outline.plot(ax=ax, facecolor="white", edgecolor='black', lw=0.3)
nx.draw_networkx_edges(G_roads, pos=nx.get_node_attributes(G_roads, "pos"), edge_color="grey", width=0.3)
nx.draw_networkx(G_tot, pos=nx.get_node_attributes(G_tot, "location"), with_labels=False, node_size=20, node_color=node_color, font_size=5, font_weight='bold', edge_color='white', arrows=False, width=0.1)
for index, row in df_edge.iterrows():
    source=row["source"]
    target=row["target"]
    source_id=dic_node_id.get(source)
    target_id=dic_node_id.get(target)
    dist_s=dic_dist.get(source)/1000
    dist_t=dic_dist.get(target)/1000
    
    if np.isnan(row["day"]):
        color_1="grey"
    else:
        i_col=int(row["day"])-20
        color_1=colors3[i_col]
    hex_color =colors.to_hex(color_1)
    shortest_path=nx.shortest_path(G_roads, source=source_id, target=target_id, weight="length")
#        nodes.plot(ax=ax, edgecolor="grey")
    path_edges = list(zip(shortest_path, shortest_path[1:]))
    nx.draw_networkx_edges(G_roads, pos=nx.get_node_attributes(G_roads, "pos"), edgelist=path_edges, edge_color=hex_color, width=2)
plt.xlim(0.1e6,1.12e6)
#plt.ylim(2.1e6,3.2e6)  
plt.savefig("map_tot_colors.pdf")
plt.show()

#nx.draw(G_tot, pos=nx.get_node_attributes(G_tot, "location"), with_labels=False, node_size=20, node_color=node_color, font_weight='bold', edge_color='white', arrows=False, width=0)




#### Figure 1a

In [ ]:
fig = plt.figure(figsize=(8,8))
ax = fig.add_axes([0, 0, 1, 1])
ax.axis('off')
fr_outline.plot(ax=ax, facecolor="white", edgecolor='black', lw=0.3)
nx.draw_networkx(G_tot, pos=nx.get_node_attributes(G_tot, "location"), with_labels=False, node_size=30, node_color=node_color, font_size=5, font_weight='bold', edge_color='black', arrows=True, width=1)
plt.xlim(0.1e6,1.12e6)
#plt.savefig("arrows_2.pdf")
plt.show()


#### Velocity distribution 

In [ ]:
list_v=[]
for u, v in G_tot.edges():
    velocity=G_tot[u][v]['velocity']
    if(pd.notna(velocity)):
        list_v.append(velocity)

In [ ]:
df_edge["road_velocity"]=df_edge["road_distance"]/df_edge["travel_time"]

In [ ]:
list_v2=df_edge.road_velocity.dropna()

In [ ]:
fig = plt.figure(figsize=(5,5))
plt.hist(list_v2, bins=np.linspace(0, 150, 25), edgecolor="tab:red", density=True, alpha=1, label="Road", histtype="step")
plt.hist(list_v, bins=np.linspace(0, 150, 25), edgecolor="tab:blue", density=True, alpha=1, label="Euclidean", histtype="step")
#plt.plot(bins[:-1],counts, ls="dashed", color="black")
plt.ylabel(r"$P(v)$")
plt.xlabel(r"$v$ (km/day)")
#plt.yscale("log")
plt.legend()

#### Travel time distribution

In [ ]:
list_t=[]
for u, v in G_tot.edges():
    time=G_tot[u][v]['travel_time']
    if(pd.notna(time)):
        list_t.append(time)

In [ ]:
fig = plt.figure(figsize=(5,5))
plt.hist(list_t, bins=[0.5, 1.5, 2.5, 3.5], edgecolor='black', align='mid', rwidth=0.8, density=True, alpha=1)
#plt.plot(bins[:-1],counts, ls="dashed", color="black")
plt.ylabel(r"$P(t)$")
plt.xlabel(r"$t$ (days)")
#plt.yscale("log")


##### Distance distribution

In [ ]:
list_d=[]
for u, v in G_tot.edges():
    distance=G_tot[u][v]['distance']
    if(pd.notna(distance)):
        list_d.append(distance)

In [ ]:
ig = plt.figure(figsize=(8,4))
plt.hist(list_d,edgecolor='black')
#plt.plot(bins[:-1],counts, ls="dashed", color="black")
plt.ylabel(r"$P(d)$")
plt.xlabel(r"$d$ (km)")
#plt.yscale("log")


##### Connected components (Fig. 1b)

In [ ]:
# Find weakly connected components
components = list(nx.weakly_connected_components(G_tot))
# Generate distinct colors for each component
colors2 = list(colors.XKCD_COLORS.keys())  # Use Tableau colors for variety
random.shuffle(colors2)
color_map = {}
for i, component in enumerate(components):
        for node in component:
            if len(component)>4:
                color_map[node] = colors2[i % len(colors2)]  # Cycle colors if components > len(colors)
            else:
              color_map[node] = "gray"  

# Assign gray to disconnected nodes
#for node in nodes:
#    if node not in color_map:
#        color_map[node] = "gray"


In [ ]:
node_colors = [color_map[node] for node in G_tot.nodes()]

In [ ]:
fig = plt.figure(figsize=(8,8))
ax = fig.add_axes([0, 0, 1, 1])
ax.axis('off')
fr_outline.plot(ax=ax, facecolor="white", edgecolor='black', lw=0.3)
#nx.draw_networkx_edges(G_roads, pos=nx.get_node_attributes(G_roads, "pos"), edge_color="grey", width=0.3)
nx.draw(G_tot, pos=nx.get_node_attributes(G_tot, "location"), node_color=node_colors, with_labels=False, node_size=30, font_weight='bold', edge_color='black', arrows=True, width=0.5)
plt.xlim(0.1e6,1.12e6)
#plt.savefig("map_clustes_arrow.pdf")
plt.show()


#### Epidemic activity

In [ ]:
time_list=df_node.Day.dropna().unique()

In [ ]:
time_list=np.sort(time_list)

In [ ]:
df_degree=pd.DataFrame(columns=["time","R0","av_k"])
for time in time_list:
    filtered_nodes = [n for n, attr in G_tot.nodes(data=True) if attr.get("Day") == time]
    k = [G_tot.nodes[n]["degree"] for n in filtered_nodes]
    av_k=np.mean(k)
    av_k2=np.mean(np.array(k)**2)
    R0=av_k2/av_k-1
    df_0=pd.DataFrame({"time":[time-20],"R0":[R0],"av_k":[av_k]})
    df_degree=pd.concat([df_degree,df_0])
#    print(time,R0,av_k)

In [ ]:
df_degree=df_degree.dropna().reset_index()

In [ ]:
#sns.lineplot(data=df_degree, x="time", y="R0")
fig = plt.figure(figsize=(5,5))
sns.scatterplot(data=df_degree, x="time", y="av_k", edgecolor="black")
plt.axhline(y=1, ls="dashed")
plt.ylabel(r"$R_t$")
plt.xlabel("Days from July 20, 1789")

##### Study of re-infection

In [ ]:
ll=(df_edge.target.value_counts()>1).sum()

In [ ]:
towns=list(df_edge.target.value_counts().head(ll).index)

In [ ]:
def to_float(x):
    try:
        return float(x)
    except (ValueError, TypeError):
        return np.nan

In [ ]:
count=0
list_r=[]
list_t=[]
list_dates=[]
list_all_dates=[]
for town in towns:
    n_inf=len(df_edge[df_edge.target==town].day.unique())
    if n_inf>1:
        list_r.append(n_inf)
        list_t.append(town)
        count=count+1
        arr=df_edge[df_edge.target==town].day.unique()
        list_all_dates.append(arr)
        clean_dates = [d for d in arr 
            if not ((isinstance(d, float) and np.isnan(d)) or 
                    (isinstance(d, str) and d.strip().lower() == "nan"))]
        clean_dates=np.sort(clean_dates)
        if len(clean_dates)>1:
            list_dates.append(clean_dates[1:])


In [ ]:
list_times=[]
for arr in list_all_dates:
    sort_arr=np.sort(arr)
    times=np.diff(sort_arr)
    list_times.append(times)

In [ ]:
flattened_times = np.concatenate(list_times)

In [ ]:
fig = plt.figure(figsize=(5,5))
plt.hist(flattened_times, bins=np.linspace(0.5,4.5,5), edgecolor="black")
plt.xlabel(r"$\Delta t$ Delay in re-infection")
plt.ylabel(r"$P(\Delta t)$ number of occurrences")

In [ ]:
# average re-infection time
np.nanmean(flattened_times)

In [ ]:
reinf_days=np.concatenate(list_dates)

In [ ]:
reinf_days=reinf_days-20

In [ ]:
activity=df_node.groupby("day").count().reset_index()[["day","name"]]
activity.day=activity.day-20

#### Comparison with the SIRI model

In [ ]:
def deriv_siri(z, t, N, beta, gamma, alpha):
# definition of the derivatives in the SIR model
    S, I, R, IR = z
    dSdt = -beta * S * (I+IR) / N
    dIdt = beta * S * (I+IR) / N - gamma * I
    dRdt = gamma * (I+IR) -alpha * R * (I+IR) / N
    dIRdt=alpha * R * (I+IR) / N-gamma*IR
    return dSdt, dIdt, dRdt, dIRdt


In [ ]:
def siri_simul(t_max,N,beta,gamma, alpha):
# simulation of the SIR model
    t = np.linspace(0, t_max, t_max)
    I0, R0, IR0 = 1, 0,0
    # Everyone else, S0, is susceptible to infection initially.
    S0 = N - I0 - R0- IR0
    z0 = S0, I0, R0, IR0
    ret = odeint(deriv_siri, z0, t, args=(N, beta, gamma, alpha))
    S, I, R, IR = ret.T
    return t,S,I,R,IR

In [ ]:
def objective_siri(params):
# Objective function for the fit
    gamma, beta, alpha = params
    t_max=18
    N=501
    t, S, I, R, IR =siri_simul(t_max,N,beta,gamma,alpha)
    simulated_I = np.cumsum(I)
    simulated_IR=np.cumsum(IR)
    # Calculate the mean squared error between data_series and simulated_values
    mse = np.mean((simulated_I - cum_F) ** 2+(simulated_IR - cum_R) ** 2)
    return mse

In [ ]:
reinfection=np.histogram(reinf_days, bins=np.linspace(0,18,19))[0]

In [ ]:
first=activity.name.to_numpy()

In [ ]:
cum_R=np.cumsum(reinfection)

In [ ]:
cum_F=np.cumsum(first)

In [ ]:
x=np.linspace(0,17,18)

In [ ]:
# Initial guess of parameters
gamma=0.8
beta=1.4
alpha=0.1
N=501
initial_guess=[gamma, beta, alpha] 

In [ ]:
# Perform the optimization
result = minimize(objective_siri, initial_guess, method='L-BFGS-B', bounds=[(0,10),(0,10),(0,10)])


In [ ]:
gamma=result.x[0]
beta=result.x[1]
alpha=result.x[2]
print(beta,gamma, alpha)

In [ ]:
t_max=18
t,S, I, R, IR=siri_simul(t_max,N,beta,gamma, alpha)

In [ ]:
fig = plt.figure(figsize=(5,5))
plt.scatter(x,cum_F, edgecolor="black", label="first")
plt.scatter(x,cum_R, edgecolor="black", label="reinf.")
simulated_I = np.cumsum(I)
simulated_IR=np.cumsum(IR)
plt.plot(t, simulated_I,  color="black", ls="dashed", lw=0.8, label="SIRI")
plt.plot(t, simulated_IR,  color="black", ls="dashed", lw=0.8)
#plt.plot(x[2:11], aa*2**(bb*x[2:11]), linestyle='dashed', color="tab:blue", lw=0.8, label=r"$C*2^{t/\tau_D}$ $\tau_D=49$h#")
#plt.yscale("log")
plt.legend()
plt.ylabel("Towns affected (cumulative)")
plt.xlabel("Days from July 20, 1789")
plt.tight_layout()
#plt.savefig("cumul.pdf")

In [ ]:
fig = plt.figure(figsize=(5,5))
plt.scatter(x, first, edgecolor="black", color="tab:blue", label="first")
plt.scatter(x, reinfection, edgecolor="black", color="tab:orange", label="reinf.")
plt.plot(t,I,color="black",  ls="dashed", lw=0.7, label="SIRI")
plt.plot(t,IR,color="black",  ls="dashed", lw=0.7)
plt.legend()
plt.ylabel("Towns affected in day")
plt.xlabel("Days from July 20, 1789")
plt.tight_layout()
#plt.savefig("peak.pdf")

#### Comparison with the SIR model

In [ ]:
def deriv(y, t, N, beta, gamma):
# definition of the derivatives in the SIR model
    S, I, R = y
    dSdt = -beta * S * I / N
    dIdt = beta * S * I / N - gamma * I
    dRdt = gamma * I
    return dSdt, dIdt, dRdt


In [ ]:
def sir_simul(t_max,N,beta,gamma):
# simulation of the SIR model
    t = np.linspace(0, t_max, t_max)
    I0, R0 = 1, 0
    # Everyone else, S0, is susceptible to infection initially.
    S0 = N - I0 - R0
    y0 = S0, I0, R0
    ret = odeint(deriv, y0, t, args=(N, beta, gamma))
    S, I, R = ret.T
    return t,S,I,R



In [ ]:
# Initial guess of parameters
gamma=0.8
beta=1.4
initial_guess=[gamma, beta] 

In [ ]:
activity=df_node.groupby("Day").count().reset_index()[["Day","name"]]
activity.Day=activity.Day-20

In [ ]:
data_series1=activity.name.to_numpy()

In [ ]:
data_series2=np.cumsum(data_series1)

In [ ]:
def objective(params):
# Objective function for the fit
    gamma, beta = params
    t_max=18
    N=501
    t, S, I, R=sir_simul(t_max,N,beta,gamma)
    simulated_values = I+R
    # Calculate the mean squared error between data_series and simulated_values
    mse = np.mean((data_series2 - simulated_values) ** 2)
    return mse

In [ ]:
# Perform the optimization
result = minimize(objective, initial_guess, method='L-BFGS-B', bounds=[(0,10),(0,10)])


In [ ]:
gamma=result.x[0]
beta=result.x[1]


In [ ]:
print(beta,gamma,N)

In [ ]:
t_max=18
t,S, I, R=sir_simul(t_max,N,beta,gamma)

In [ ]:
x=activity.Day
y=activity["name"].cumsum()

In [ ]:
fig = plt.figure(figsize=(5,5))
plt.scatter(x,y, edgecolor="black", label="data")
plt.plot(t,R+I, color="tab:blue", ls="dashed", lw=0.8, label="SIR")
#plt.plot(x[2:11], aa*2**(bb*x[2:11]), linestyle='dashed', color="tab:blue", lw=0.8, label=r"$C*2^{t/\tau_D}$ $\tau_D=49$h")
plt.yscale("log")
plt.legend()
plt.ylabel("Towns affected (cumulative)")
plt.xlabel("Days from July 20, 1789")
plt.tight_layout()
#plt.savefig("cumul.pdf")


In [ ]:
diff=np.gradient(I+R)

In [ ]:
fig = plt.figure(figsize=(5,5))
plt.scatter(x,data_series1, edgecolor="black", color="red", label="data")
plt.plot(t,diff,color="black",  ls="dashed", lw=0.7, label="SIR")
plt.legend()
plt.ylabel("Towns affected in day")
plt.xlabel("Days from July 20, 1789")

### Post-relays

In [ ]:
df_node['geometry'] = gpd.points_from_xy([loc[0] for loc in df_node['location']],
                                    [loc[1] for loc in df_node['location']])

In [ ]:
gdf_node=gpd.GeoDataFrame(df_node, geometry="geometry")

In [ ]:
gdf_node=gdf_node.set_crs("EPSG:2154")


In [ ]:
# download file from: https://zenodo.org/records/11196161
layers = fiona.listlayers("../Maps/post-office/Post_Roads.gpkg")
print("Available layers:", layers)

In [ ]:
gdf_outline = gpd.read_file("../Maps/post-office/Post_Roads.gpkg", layer="France_1783")

In [ ]:
gdf_roads = gpd.read_file("../Maps/post-office/Post_Roads.gpkg", layer="roads_1783")

In [ ]:
gdf_relays=gpd.read_file("../Maps/post-office/Post_Roads.gpkg", layer="relays_1783")

In [ ]:
# Buffer the relay points by 5000 meters (5 km)
gdf_relay_buffered = gdf_relays.copy()
gdf_relay_buffered["geometry"] = gdf_relay_buffered.geometry.buffer(5000)
# Spatial join: find which nodes fall within any buffered relay zone
joined = gpd.sjoin(gdf_node, gdf_relay_buffered, how="inner", predicate="intersects")


In [ ]:
# Count unique nodes in gdf_node that are within 5 km of any gdf_relay
count_within_5km = joined["index_right"].nunique()

In [ ]:
# Spatial join: mark which nodes fall in any relay buffer
gdf_node["in_relay"] = gdf_node.geometry.apply(
    lambda geom: gdf_relay_buffered.geometry.intersects(geom).any()
)

In [ ]:
fig = plt.figure(figsize=(8,8))
ax = fig.add_axes([0, 0, 1, 1])
ax.axis('off')
gdf_roads.plot(ax=ax, color="green", lw=1)
fr_outline.plot(ax=ax, facecolor="white", edgecolor='black', lw=0.3)
#gpd.GeoSeries(fr_outline).boundary.plot(ax=ax, edgecolor='black', lw=0.3)
nx.draw(G_tot, pos=nx.get_node_attributes(G_tot, "location"), node_color="black", with_labels=False, node_size=0,  font_size=5, font_weight='bold', edge_color='black', arrows=True, width=1)
gdf_node.plot(ax=ax, column="in_relay", cmap="YlGn", edgecolor="black", s=40)
#gpd_post.plot(ax=ax, color="green")
plt.xlim(0.1e6,1.12e6)
#plt.savefig("post_2.pdf")
plt.show()

#### Movie S1

In [ ]:
list_Days=list(df_node["day"].dropna().unique())



In [ ]:
list_Days=np.sort(list_Days)

In [ ]:
for day in list_Days:
    fig = plt.figure(figsize=(8,8))
    ax = fig.add_axes([0, 0, 1, 1])
    ax.axis('off')
    fr_outline.plot(ax=ax, facecolor="white", edgecolor='black', lw=0.3)
    nodes_with_day_x = [node for node, attr in G_tot.nodes(data=True) if attr.get("day") == day]
    nodes_less_day_x = [node for node, attr in G_tot.nodes(data=True) if attr.get("day") is not None and not math.isnan(attr.get("day", np.nan)) and attr["day"] <= day]
# Create a subgraph with these nodes
    subgraph1 = G_tot.subgraph(nodes_less_day_x)
    subgraph = G_tot.subgraph(nodes_with_day_x)
    nx.draw(subgraph1, pos=nx.get_node_attributes(subgraph1, "location"), node_color="grey", with_labels=False, node_size=40, edge_color='white', arrows=False)
    nx.draw(subgraph, pos=nx.get_node_attributes(subgraph, "location"), node_color="red", with_labels=False, node_size=40, edge_color='white', arrows=False)
    plt.xlim(0.1e6,1.12e6)
    plt.savefig("../MovieS1/progress"+str(day)+".png")
    plt.show()
    
    

#### Movie S2

In [ ]:
for day in list_Days:
    fig = plt.figure(figsize=(8,8))
    ax = fig.add_axes([0, 0, 1, 1])
    ax.axis('off')
    fr_outline.plot(ax=ax, facecolor="white", edgecolor='black', lw=0.3)
    nx.draw_networkx_edges(G_roads, pos=nx.get_node_attributes(G_roads, "pos"), edge_color="grey", width=0.3)
    for index, row in df_edge.iterrows():
        if not np.isnan(row["day"]) and row["day"]<=day:
            source=row["source"]
            target=row["target"]
            source_id=dic_node_id.get(source)
            target_id=dic_node_id.get(target)
            dist_s=dic_dist.get(source)/1000
            dist_t=dic_dist.get(target)/1000
            color="tab:blue"
#            (x1,y1)=ast.literal_eval(row["origin_loc"])
#            (x2,y2)=ast.literal_eval(row["destination_loc"])
#            plt.scatter(x1,y1, color=color)
#            plt.scatter(x2,y2, color="red")
            shortest_path=nx.shortest_path(G_roads, source=source_id, target=target_id, weight="length")
#        nodes.plot(ax=ax, edgecolor="grey")
            path_edges = list(zip(shortest_path, shortest_path[1:]))
            nx.draw_networkx_edges(G_roads, pos=nx.get_node_attributes(G_roads, "pos"), edgelist=path_edges, edge_color=color, width=1.5)  
    nodes_with_day_x = [node for node, attr in G_tot.nodes(data=True) if attr.get("day") == day]
#    nodes_less_day_x = [node for node, attr in G_tot.nodes(data=True) if attr.get("day") is not None and not math.isnan(attr.get("day", np.nan)) and attr["day"] <= day]
# Create a subgraph with these nodes
#    subgraph1 = G_tot.subgraph(nodes_less_day_x)
    subgraph = G_tot.subgraph(nodes_with_day_x)
#    nx.draw(subgraph1, pos=nx.get_node_attributes(subgraph1, "location"), node_color="tab:blue", with_labels=False, node_size=40, edge_color='white', arrows=False)
    nx.draw(subgraph, pos=nx.get_node_attributes(subgraph, "location"), node_color="red", with_labels=False, node_size=40, edge_color='white', arrows=False)
    plt.xlim(0.1e6,1.12e6)
    plt.savefig("../MovieS2/roads"+str(day)+".png")
    plt.show()
    
    